# PSyKE's demo for regression comparison

Some imports.

In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from os.path import exists
from psyke import Extractor
from psyke.utils.logic import pretty_theory
from psyke.regression import FeatureRanker
from psyke.optimizer.pedro import PEDRO
from psyke.optimizer.crash import CRASH
from psyke.utils import Target
from psyke.optimizer import Objective

import warnings
warnings.simplefilter("ignore")

Loading black-box models

In [2]:
PATH = "../test/resources/"
datasets = ["contingency", "contingency", "anticipate", "anticipate", "contingency", "anticipate"]
models = [
    "CONTINGENCY_no_input-memory_DecisionTree_MaxDepth10",
    "CONTINGENCY_no_input-time_DecisionTree_MaxDepth10",
    "ANTICIPATE_no_input-memory_DecisionTree_MaxDepth10",
    "ANTICIPATE_no_input-time_DecisionTree_MaxDepth10",
    "CONTINGENCY_input-cost_DecisionTree_MaxDepth15",
    "ANTICIPATE_input-cost_DecisionTree_MaxDepth15"
]
models = [
    pickle.load(open(f"{PATH}predictors/{dataset}/{path}", 'rb')) for path, dataset in zip(models, datasets)
]

Function to pre-process data

In [3]:
def process(dataset):
    df = pd.read_csv(f"{PATH}datasets/{dataset}_trainDataset.csv")

    # Removes header entries
    df = df[df['sol(keuro)'] != 'sol(keuro)']

    # Fixed stuff which is always there
    df['PV(kW)'] = df['PV(kW)'].map(lambda entry: entry[1:-1].split())
    df['PV(kW)'] = df['PV(kW)'].map(lambda entry: list(np.float_(entry)))
    df['Load(kW)'] = df['Load(kW)'].map(lambda entry: entry[1:-1].split())
    df['Load(kW)'] = df['Load(kW)'].map(lambda entry: list(np.float_(entry)))

    X = pd.DataFrame()

    X['PV_mean'] = df['PV(kW)'].map(lambda entry: np.array(entry).mean())
    X['PV_std'] = df['PV(kW)'].map(lambda entry: np.array(entry).std())
    X['Load_mean'] = df['Load(kW)'].map(lambda entry: np.array(entry).mean())
    X['Load_std'] = df['Load(kW)'].map(lambda entry: np.array(entry).std())
    X['nScenarios'] = df['nScenarios']
    X['cost'] = df['sol(keuro)']
    X['time'] = df['time(sec)']
    X['memo'] = df['memAvg(MB)']

    X.to_csv(f"{PATH}datasets/{dataset}.csv", index = False)

    return X

Experiment setting

In [4]:
toRemove = [
    ['PV_mean', 'PV_std', 'Load_mean', 'Load_std', 'time', 'cost'],
    ['PV_mean', 'PV_std', 'Load_mean', 'Load_std', 'memo', 'cost'],
    ['PV_mean', 'PV_std', 'Load_mean', 'Load_std', 'time', 'cost'],
    ['PV_mean', 'PV_std', 'Load_mean', 'Load_std', 'memo', 'cost'],
    ["time", "memo"],
    ["time", "memo"]
]

features = [
    ["nTraces"],
    ["nTraces"],
    ["nScenarios"],
    ["nScenarios"],
    ['PV_mean', 'PV_std', 'Load_mean', 'Load_std', 'nTraces'],
    ['PV_mean', 'PV_std', 'Load_mean', 'Load_std', 'nScenarios']
]

targets = ["memo", "time", "memo", "time", "cost", "cost"]

Knowledge extraction with CReEPy, GridREx, GridEx and CART

Automated selection of the best hyper-parameters for GridREx and GridEx with PEDRO algorithm.
For CReEPy we use CRASH, a work-in-progress procedure to tune its parameters.

In [5]:
best_params = {
    'GridREx': [],
    'GridEx': [],
    'CReEPy': []
}

for rem, feat, target, dataset, model in zip(toRemove, features, targets, datasets, models):
    print(f"Dataset: {dataset}, Output variable: {target}, Input features: {len(feat)}")
    name = f"{PATH}datasets/{dataset}.csv"

    if not exists(name):
        process(dataset)

    dataset = pd.read_csv(name).drop(rem, axis = 1)
    ranked = FeatureRanker(dataset.columns[:-1]).fit(model, dataset.iloc[:, :-1]).rankings()

    train, test = train_test_split(dataset, test_size=0.2, random_state=10)
    model.fit(train.iloc[:, :-1], train.iloc[:, -1])

    print("\n============GridREx parameter auto-tuning with PEDRO============\n")
    pedro = PEDRO(model, train, max_mae_increase=1.2, min_rule_decrease=0.9, readability_tradeoff=0.1,
                  max_depth=2, patience=1, algorithm=PEDRO.Algorithm.GRIDREX, objective=Objective.MODEL)
    pedro.search()
    best_params['GridREx'].append(pedro.get_best()[0])

    print("\n============GridEx parameter auto-tuning with PEDRO============\n")
    pedro = PEDRO(model, train, max_mae_increase=1.2, min_rule_decrease=0.9, readability_tradeoff=0.1,
                  max_depth=2, patience=1, algorithm=PEDRO.Algorithm.GRIDEX, objective=Objective.MODEL)
    pedro.search()
    best_params['GridEx'].append(pedro.get_best()[0])

    print("\n============CReEPy parameter auto-tuning with CRASH============\n")
    crash = CRASH(model, train, readability_tradeoff=0.1, max_depth=5, patience=1,
                  algorithm=CRASH.Algorithm.CReEPy, objective=Objective.MODEL)
    crash.search()
    best_params['CReEPy'].append(crash.get_best()[0])

    print("\n*************************************************************************************\n")

Dataset: contingency, Output variable: memo, Input features: 1

============GridREx parameter auto-tuning with PEDRO============

Algorithm.GRIDREX. Grid (1). Fixed (2). Threshold = 1.91. MAE = 2.33, 2 rules
Algorithm.GRIDREX. Grid (1). Fixed (2). Threshold = 3.82. MAE = 2.31, 1 rules

Algorithm.GRIDREX. Grid (2). Fixed (2). Threshold = 1.91. MAE = 2.31, 3 rules
Algorithm.GRIDREX. Grid (2). Fixed (2). Threshold = 3.82. MAE = 2.31, 1 rules

Algorithm.GRIDREX. Grid (1). Fixed (3). Threshold = 1.91. MAE = 2.33, 2 rules
Algorithm.GRIDREX. Grid (1). Fixed (3). Threshold = 3.82. MAE = 2.31, 1 rules

Algorithm.GRIDREX. Grid (2). Fixed (3). Threshold = 1.91. MAE = 2.29, 4 rules
Algorithm.GRIDREX. Grid (2). Fixed (3). Threshold = 3.82. MAE = 2.31, 1 rules

Algorithm.GRIDREX. Grid (1). Adaptive ([(0.99, 5)]). Threshold = 1.91. MAE = 2.32, 2 rules
Algorithm.GRIDREX. Grid (1). Adaptive ([(0.99, 5)]). Threshold = 3.82. MAE = 2.31, 1 rules

Algorithm.GRIDREX. Grid (2). Adaptive ([(0.99, 5)]). Thresh

Evaluation of the extractors trained with the best hyper-parameters.
Only 5 executions per algorithm per dataset, since they generally provide the same results

In [8]:
def evaluate(extractor, dataset):
    theory = extractor.extract(dataset)
    n = extractor.n_rules
    print(f'{n} rules. ', end='')
    mae = extractor.mae(test)
    print(f'MAE (data) = {mae:.2f}. ', end='')
    fid = extractor.mae(test, model)
    print(f'MAE (BB) = {fid:.2f}')
    return theory, n, mae, fid

In [14]:
averaged = {
    'GridREx': [],
    'GridEx': [],
    'CReEPy': [],
    'CART': []
}

theories = {
    'GridREx': [],
    'GridEx': [],
    'CReEPy': [],
    'CART': []
}

max_executions = 5

for i, (rem, feat, target, dataset, model) in enumerate(zip(toRemove, features, targets, datasets, models)):
    results = {
        'GridREx': [],
        'GridEx': [],
        'CReEPy': [],
        'CART': []
    }

    print(f"Dataset: {dataset}, Output variable: {target}, Input features: {len(feat)}")

    dataset = pd.read_csv(f"{PATH}datasets/{dataset}.csv").drop(rem, axis = 1)

    train, test = train_test_split(dataset, test_size=0.2, random_state=10)
    model.fit(train.iloc[:, :-1], train.iloc[:, -1])
    E = abs(model.predict(test.iloc[:, :-1]) - test.iloc[:, -1])
    print(f"BB MAE = {E.mean():.2f} +- {E.std():.2f}")

    print("\n============GridREx============\n")
    for j in range(max_executions):
        print(f"Execution #{j + 1}. ", end='')
        gridREx = Extractor.gridrex(model, best_params['GridREx'][i][3], threshold=best_params['GridREx'][i][2])
        theory_from_gridREx, n, mae, fid = evaluate(gridREx, train)
        results['GridREx'].append([n, mae, fid])

    print("\n============GridEx============\n")
    for j in range(max_executions):
        print(f"Execution #{j + 1}. ", end='')
        gridEx = Extractor.gridex(model, best_params['GridEx'][i][3], threshold=best_params['GridEx'][i][2])
        theory_from_gridEx, n, mae, fid = evaluate(gridEx, train)
        results['GridEx'].append([n, mae, fid])

    print("\n============CReEPy============\n")
    for j in range(max_executions):
        print(f"Execution #{j + 1}. ", end='')
        creepy = Extractor.creepy(model, depth=best_params['CReEPy'][i][2],
                                  error_threshold=best_params['CReEPy'][i][3], output=Target.REGRESSION)
        theory_from_creepy, n, mae, fid = evaluate(creepy, train)
        results['CReEPy'].append([n, mae, fid])

    print("\n============CART============\n")
    for j in range(max_executions):
        print(f"Execution #{j + 1}. ", end='')
        cart = Extractor.cart(model, max_leaves=4, simplify=True)
        theory_from_cart, n, mae, fid = evaluate(cart, train)
        results['CART'].append([n, mae, fid])

    print("\n*************************************************************************************\n")

    for extractor, theory in zip(['GridREx', 'GridEx', 'CReEPy', 'CART'],
                                 [theory_from_gridREx, theory_from_gridEx, theory_from_creepy, theory_from_cart]):
        res = np.array(results[extractor]).mean(axis=0)
        std = np.array(results[extractor]).std(axis=0)
        averaged[extractor].append((res, std))
        theories[extractor].append(theory)

Dataset: contingency, Output variable: memo, Input features: 1
BB MAE = 3.64 +- 4.07

============GridREx============

Execution #1. 4 rules. MAE (data) = 4.66. MAE (BB) = 2.30
Execution #2. 4 rules. MAE (data) = 4.66. MAE (BB) = 2.30
Execution #3. 4 rules. MAE (data) = 4.66. MAE (BB) = 2.30
Execution #4. 4 rules. MAE (data) = 4.66. MAE (BB) = 2.30
Execution #5. 4 rules. MAE (data) = 4.66. MAE (BB) = 2.30

============GridEx============

Execution #1. 4 rules. MAE (data) = 4.68. MAE (BB) = 2.31
Execution #2. 4 rules. MAE (data) = 4.68. MAE (BB) = 2.31
Execution #3. 4 rules. MAE (data) = 4.68. MAE (BB) = 2.31
Execution #4. 4 rules. MAE (data) = 4.68. MAE (BB) = 2.31
Execution #5. 4 rules. MAE (data) = 4.68. MAE (BB) = 2.31

============CReEPy============

Execution #1. 2 rules. MAE (data) = 4.65. MAE (BB) = 2.31
Execution #2. 2 rules. MAE (data) = 4.65. MAE (BB) = 2.31
Execution #3. 2 rules. MAE (data) = 4.65. MAE (BB) = 2.31
Execution #4. 2 rules. MAE (data) = 4.65. MAE (BB) = 2.31
Exe

Aggregated results

In [20]:
for i, (feat, target, dataset) in enumerate(zip(features, targets, datasets)):
    print(f"Dataset: {dataset}, Output variable: {target}, Input features: {len(feat)}\n")

    for extractor in ['GridREx', 'GridEx', 'CReEPy', 'CART']:
        res, std = averaged[extractor][i]
        print(f'{extractor} average data. {int(res[0])} rules, '
              f'MAE (data) = {res[1]:.2f} +- {std[0]:.2f}, '
              f'MAE (BB) = {res[2]:.2f} +- {std[0]:.2f}.')
    print("\n*************************************************************************************\n")

Dataset: contingency, Output variable: memo, Input features: 1

GridREx average data. 4 rules, MAE (data) = 4.66 +- 0.00, MAE (BB) = 2.30 +- 0.00.
GridEx average data. 4 rules, MAE (data) = 4.68 +- 0.00, MAE (BB) = 2.31 +- 0.00.
CReEPy average data. 2 rules, MAE (data) = 4.65 +- 0.00, MAE (BB) = 2.31 +- 0.00.
CART average data. 4 rules, MAE (data) = 4.52 +- 0.00, MAE (BB) = 2.13 +- 0.00.

*************************************************************************************

Dataset: contingency, Output variable: time, Input features: 1

GridREx average data. 5 rules, MAE (data) = 0.97 +- 0.00, MAE (BB) = 0.67 +- 0.00.
GridEx average data. 5 rules, MAE (data) = 3.14 +- 0.00, MAE (BB) = 3.06 +- 0.00.
CReEPy average data. 2 rules, MAE (data) = 1.00 +- 0.00, MAE (BB) = 0.74 +- 0.00.
CART average data. 4 rules, MAE (data) = 3.92 +- 0.00, MAE (BB) = 3.76 +- 0.00.

*************************************************************************************

Dataset: anticipate, Output variable: memo

Example of output theories for each dataset for each extractor

In [21]:
for i, (feat, target, dataset) in enumerate(zip(features, targets, datasets)):
    print(f"Dataset: {dataset}, Output variable: {target}, Input features: {len(feat)}")

    for extractor in ['GridREx', 'GridEx', 'CReEPy', 'CART']:
        print(f"\n============{extractor}============\n")
        print(f'{extractor} extracted rules (last execution):\n' + pretty_theory(theories[extractor][i]))
    print("\n*************************************************************************************\n")

Dataset: contingency, Output variable: memo, Input features: 1

============GridREx============

GridREx extracted rules (last execution):
memo(NTraces, Memo) :-
    NTraces in [33.99, 100.00], Memo is 88.22 + 0.03 * NTraces.
memo(NTraces, Memo) :-
    NTraces in [0.99, 11.99], Memo is 87.12 + -0.04 * NTraces.
memo(NTraces, Memo) :-
    NTraces in [11.99, 22.99], Memo is 80.77 + 0.40 * NTraces.
memo(NTraces, Memo) :-
    NTraces in [22.99, 33.99], Memo is 82.42 + 0.20 * NTraces.

============GridEx============

GridEx extracted rules (last execution):
memo(NTraces, 87.21) :-
    NTraces in [0.99, 25.74].
memo(NTraces, 90.13) :-
    NTraces in [25.74, 50.5].
memo(NTraces, 89.07) :-
    NTraces in [50.5, 75.25].
memo(NTraces, 91.41) :-
    NTraces in [75.25, 100.00].

============CReEPy============

CReEPy extracted rules (last execution):
memo(NTraces, Memo) :-
    NTraces in [30.99, 100.00], Memo is 87.58 + 0.04 * NTraces.
memo(NTraces, Memo) :-
    NTraces in [0.99, 100.00], Memo is 8